In [765]:
import pandas as pd
from sqlalchemy import create_engine,  MetaData, Table
from datetime import date
import datetime

import json
import re

## Ouverture des csv en dataframe

### aeronefs

In [766]:
# ouvrir csv pour mettre en data frame
df_aeronef = pd.read_csv(r'dataset\aeronefs_clean.csv')

In [767]:
df_aeronef

,ref_aero,type_model,debut_service,last_maint,en_maintenance,end_maint
0,E170_6356,E170,2000-03-17,2024-03-27,False,NaN
1,B767_2803,B767,2020-12-03,2024-06-01,False,NaN
2,CRJ700_6943,CRJ700,2015-01-29,2024-06-02,True,2024-06-03 14:13:00
3,A350_1177,A350,2005-08-07,2024-01-04,False,NaN
4,B777_4029,B777,2009-01-01,2024-06-01,False,NaN
...,...,...,...,...,...,...
226,B737_4369,B737,2010-01-05,2024-06-01,False,NaN
227,A340_4021,A340,2004-12-13,2024-05-31,False,NaN
228,CRJ900_1740,CRJ900,2003-07-20,2024-06-01,False,NaN
229,E170_5551,E170,2021-04-10,2024-06-01,False,NaN


In [768]:
# convertir les données avant insertion dans la table sql
df_aeronef['debut_service'] = pd.to_datetime(df_aeronef['debut_service'])
df_aeronef['last_maint'] = pd.to_datetime(df_aeronef['last_maint'])
df_aeronef['end_maint'] = pd.to_datetime(df_aeronef['end_maint'])

df_aeronef['en_maintenance'] = df_aeronef['en_maintenance'].astype('boolean')


In [769]:
df_aeronef

,ref_aero,type_model,debut_service,last_maint,en_maintenance,end_maint
0,E170_6356,E170,2000-03-17,2024-03-27,False,NaT
1,B767_2803,B767,2020-12-03,2024-06-01,False,NaT
2,CRJ700_6943,CRJ700,2015-01-29,2024-06-02,True,2024-06-03 14:13:00
3,A350_1177,A350,2005-08-07,2024-01-04,False,NaT
4,B777_4029,B777,2009-01-01,2024-06-01,False,NaT
...,...,...,...,...,...,...
226,B737_4369,B737,2010-01-05,2024-06-01,False,NaT
227,A340_4021,A340,2004-12-13,2024-05-31,False,NaT
228,CRJ900_1740,CRJ900,2003-07-20,2024-06-01,False,NaT
229,E170_5551,E170,2021-04-10,2024-06-01,False,NaT


### Composants

In [770]:
# ouvrir csv pour mettre en data frame
df_composants = pd.read_csv(r'dataset\composants_clean.csv')

In [771]:
# convertir les données avant insertion dans la table sql

# -> pas de besoin


In [772]:
df_composants.head()

,ref_compo,categorie,aero,desc_compo,lifespan,taux_usure_actuel,cout
0,REAE170-E170_6353-0,Composants Critiques,E170_6353,Réacteur gauche,13045,53.308375,15552
1,REAE170-E170_6353-1,Composants Critiques,E170_6353,Réacteur droit,13626,50.111950,17559
2,SYSE170-E170_6353-2,Composants Critiques,E170_6353,Système de navigation,10541,16.730575,16514
3,ORDE170-E170_6353-3,Composants Critiques,E170_6353,Ordinateur de vol,10441,6.653075,16047
4,AUTE170-E170_6353-4,Composants Critiques,E170_6353,Autopilote,14847,42.033525,19362


### degradations

In [773]:
# cration date du jour
aujourdhui = date.today()
aujourdhui = aujourdhui.strftime('%Y-%m-%d')

# aujourdhui = '2024-07-23'

In [774]:
# ouvrir csv pour mettre en data frame
df_degradations = pd.read_csv(f'http://sc-e.fr/docs/degradations_{aujourdhui}.csv')

In [775]:
df_degradations.head()

,ref_deg,linked_aero,compo_concerned,usure_nouvelle,measure_day,need_replacement
0,D003661,E170_6353,REAE170-E170_6353-0,74.467365,2024-07-22,False
1,D009402,E170_6353,REAE170-E170_6353-1,72.213322,2024-07-22,False
2,D000812,E170_6353,SYSE170-E170_6353-2,33.828077,2024-07-22,False
3,D006665,E170_6353,ORDE170-E170_6353-3,23.588377,2024-07-22,False
4,D001163,E170_6353,AUTE170-E170_6353-4,66.115359,2024-07-22,False


#### preparation des données / nettoyage

In [776]:
# Ajout d'une colonne pour primaryKey
df_degradations.insert(0, 'clef', df_degradations.apply(lambda row: str(row['ref_deg']) + str(row['linked_aero'])  + str(row['compo_concerned'])+ str(row['measure_day']), axis=1))


In [777]:
#modification du type de la colonne date

df_degradations['measure_day'] = df_degradations['measure_day'].str.split().str[0]
df_degradations['measure_day'] = pd.to_datetime(df_degradations['measure_day'], format='%Y-%m-%d')


### logs_vols

In [778]:
# ouvrir csv pour mettre en data frame
df_logs_vols = pd.read_csv(f'http://sc-e.fr/docs/logs_vols_{aujourdhui}.csv')

In [779]:
#modification du type de la colonne date

df_logs_vols['jour_vol'] = df_logs_vols['jour_vol'].str.split().str[0]
df_logs_vols['jour_vol'] = pd.to_datetime(df_logs_vols['jour_vol'], format='%Y-%m-%d')

In [780]:
# séparation de la colonne sensor_data en plusieurs colonne
# 1- regarder les diffrent sensor 

list_sensor = list()

for elem in df_logs_vols['sensor_data']:

    elem = json.loads(elem.replace('\'', '"'))

    for key in elem.keys():
        if key not in list_sensor:
            list_sensor.append(key)

list_sensor = set(list_sensor)

list_sensor

{'pressure', 'temp', 'vibrations'}

In [781]:
# découpage des sensor en colonne en supprimant les unités de la colonne et ajoutant une col unité

#patternn de séparation des unités
pattern = r"([-+]?\d*\.?\d+)((°C)?( hPa)?( m/s²)?)"

for sensor in list_sensor:
    df_logs_vols[sensor] = df_logs_vols['sensor_data'].apply(lambda x: re.match(pattern, json.loads(x.replace('\'', '"')).get(sensor)).group(1))
    df_logs_vols[f'{sensor}_unit'] = df_logs_vols['sensor_data'].apply(lambda x: re.match(pattern, json.loads(x.replace('\'', '"')).get(sensor)).group(2))


In [782]:
# passage des variables des sensors en float

for sensor in list_sensor:
    df_logs_vols[sensor] = df_logs_vols[sensor].astype(float)

In [783]:
df_logs_vols.head()

,ref_vol,aero_linked,jour_vol,time_en_air,sensor_data,etat_voyant,vibrations,vibrations_unit,pressure,pressure_unit,temp,temp_unit
0,V09194923,E170_0081,2024-07-23,6.6,"{'temp': '-10.0°C', 'pressure': '932.0 hPa', '...",0,3.773418,m/s²,932.0,hPa,-10.0,°C
1,V07241227,B747_6747,2024-07-23,6.6,"{'temp': '1.1°C', 'pressure': '963.7 hPa', 'vi...",0,2.634974,m/s²,963.7,hPa,1.1,°C
2,V01443548,A321_5206,2024-07-23,4.3,"{'temp': '-14.2°C', 'pressure': '969.5 hPa', '...",1,0.121416,m/s²,969.5,hPa,-14.2,°C
3,V06352606,B747_4152,2024-07-23,5.1,"{'temp': '12.7°C', 'pressure': '956.8 hPa', 'v...",0,1.909083,m/s²,956.8,hPa,12.7,°C
4,V04714204,B747_0472,2024-07-23,7.3,"{'temp': '-3.7°C', 'pressure': '928.2 hPa', 'v...",0,0.809483,m/s²,928.2,hPa,-3.7,°C


## Connexion et insertion des données dans la bdd

### Connexion à la bdd

In [784]:
# DÃ©finition des paramÃ¨tres de connexion Ã  la base de donnÃ©es
username = 'postgres'
password = 'admin'
host = 'localhost'
port = '5432'
dbname = 'preventive_maintenance'

# Configuration de la connexion Ã  la base de donnÃ©es PostgreSQL
connection_str = f'postgresql+psycopg2://{username}:{password}@{host}:{port}/{dbname}'
engine = create_engine(connection_str, echo = False)

# generation de l'obet bdd
metadata = MetaData()
metadata.reflect(bind=engine)


### declaration des tables

In [785]:
aeronefs = Table(
    'aeronefs', metadata,
    )

composants = Table(
    'composants', metadata,
    )

degradations = Table(
    'degradations', metadata,
    )

logs_vols = Table(
    'logs_vols', metadata,
    )

aeronefs_cdc = Table(
    'aeronefs_cdc', metadata,
    )

composants_cdc = Table(
    'composants_cdc', metadata,
    )

degradations_cdc = Table(
    'degradations_cdc', metadata,
    )

logs_vols_cdc = Table(
    'logs_vols_cdc', metadata,
    )

metadata.create_all(engine)

### insertion des données

In [786]:

def maj_table(df:pd.DataFrame, table : Table, table_cdc : Table)->None:  # , ref_temp: str, data_to:dict[str:str, str:Date, str:int]
    """
    InsÃ¨re ou met à jour les donnÃ©es d'une table SQL à partir d'un DataFrame Pandas.

    Pour chaque ligne du DataFrame, la fonction vÃ©rifie si la rÃ©fÃ©rence (ref_aero) existe déjà dans la table.
    Si la rÃ©fÃ©rence n'existe pas, la ligne est insÃ©rÃ©e. Si la rÃ©fÃ©rence existe et que les donnÃ©es sont diffÃ©rentes,
    la ligne est mise à jour.

    Parameters
    ----------
    df : pd.DataFrame
        DataFrame contenant les donnÃ©es à insÃ©rer ou mettre à jour dans la table.
    table : Table
        Objet Table de SQLAlchemy reprÃ©sentant la table dans laquelle les donnÃ©es seront insÃ©rÃ©es ou mises à jour.
    """   
    with engine.connect() as connection:
        
        comptage_insertion = 0
        comptage_update = 0

        # Parcour de l'ensemble des lignes du df à  inserer
        for row in df.itertuples(index=False):

            data_to = {}
            
            # Préparation des données à  inserer ou updater
            column_names = df.columns

            for index, col in zip(range(len(row)), column_names):

                if row[index]  is not pd.NaT:
                    data_to[col] = row[index]
                else : 
                    data_to[col] = None


            primary_key_columns = table.primary_key.columns.keys()
            
            # Récupération de la valeur de la colonne reference/primarykey
            # Verification si la clé existe dans le dataframe
            if primary_key_columns[0] in data_to:
                ref_temp = data_to[primary_key_columns[0]]
            else : 
                break

            # identification si la ref est déjà  prÃ©sente ou non dans la table 
            sel = table.select().where(table.c[primary_key_columns[0]] == ref_temp)
            result = connection.execute(sel)   
            row_in_table = result.fetchone()    

            # # identification si les infos sont dejà  existantes dans la table cdc 
            # sel = table_cdc.select().where(table_cdc.c[primary_key_columns[0]] == ref_temp)
            # result = connection.execute(sel)   
            # row_in_table = result.fetchone()   

            if row_in_table is None: 
                
                #insertion de la ligne dans la table
                connection.execute(table.insert().values(data_to))
                comptage_insertion += 1

            else:
                # mise au meme format de row et row_in_table pour les dates
                row1 = [i for i in row]
                row2 = [i for i in row_in_table]

                for i, elem in enumerate(row2):

                    if isinstance(elem, datetime.date) or elem is None:

                        row2[i] = pd.Timestamp(row2[i])

                # Ajout de la condition de doublon ou non
                if row1 != row2 and row1[0] == row2[0]:
                    # update de la ligne dans la table si non doublon
                    connection.execute(table.update().where(table.c[primary_key_columns[0]] == ref_temp).values(data_to))
                    # Information sur la mise à jour
                    print(f'Comparaison des données pour MàJ : \n{row1}\n{row2}\n')
                    
                    comptage_update += 1
                
            # Commit dans la table des insertions ou update 
            connection.commit()

        # Affichage des rÃ©sultats
        print(f'Nombre de lignes insérées : {comptage_insertion}')
        print(f'Nombre de lignes mises à jour : {comptage_update}')




In [787]:
# maj_table(df_aeronef, aeronefs, aeronefs_cdc)

In [788]:
# maj_table(df_composants, composants, composants_cdc)

In [789]:
maj_table(df_degradations, degradations, degradations_cdc)

Nombre de lignes insérées : 0
Nombre de lignes mises à jour : 0


In [790]:
maj_table(df_logs_vols, logs_vols, logs_vols_cdc)

Comparaison des données pour MàJ : 
['V09194923', 'E170_0081', Timestamp('2024-07-23 00:00:00'), 6.6, "{'temp': '-10.0°C', 'pressure': '932.0 hPa', 'vibrations': '3.773417758644968 m/s²'}", 0, 3.773417758644968, ' m/s²', 932.0, ' hPa', -10.0, '°C']
['V09194923', 'E170_0081', Timestamp('2024-07-23 00:00:00'), 6.6, "{'temp': '-10.0°C', 'pressure': '932.0 hPa', 'vibrations': '3.773417758644968 m/s²'}", 0, -10.0, '°C', 932.0, ' hPa', 3.773417758644968, ' m/s²']

Comparaison des données pour MàJ : 
['V07241227', 'B747_6747', Timestamp('2024-07-23 00:00:00'), 6.6, "{'temp': '1.1°C', 'pressure': '963.7 hPa', 'vibrations': '2.634973793033369 m/s²'}", 0, 2.634973793033369, ' m/s²', 963.7, ' hPa', 1.1, '°C']
['V07241227', 'B747_6747', Timestamp('2024-07-23 00:00:00'), 6.6, "{'temp': '1.1°C', 'pressure': '963.7 hPa', 'vibrations': '2.634973793033369 m/s²'}", 0, 1.1, '°C', 963.7, ' hPa', 2.634973793033369, ' m/s²']

Comparaison des données pour MàJ : 
['V01443548', 'A321_5206', Timestamp('2024-07-